# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [9]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [10]:
select_relevant_links("https://huggingface.co")

{'links': [{'type': 'company page', 'url': 'https://huggingface.co'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://github.com/huggingface'},
  {'type': 'company page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [ ]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [11]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [12]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-OCR
Updated
4 days ago
•
204k
•
743
moonshotai/Kimi-K2.5
Updated
2 days ago
•
335k
•
1.8k
Qwen/Qwen3-Coder-Next
Updated
4 days ago
•
53.5k
•
544
stepfun-ai/Step-3.5-Flash
Updated
about 3 hours ago
•
12k
•
493
circlestone-labs/Anima
Updated
6 days ago
•
60.6k
•
484
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.28k
Qwen3-TTS Demo
🎙
1.28k
Transform text into natural-sounding speech with custom voices
Running
on
A100
162
ACE-Step v1.5
🎵
162
Music Generation Foundation Model v1.5
Running
463
Demo Playground
⚡
463
Free platform to access multiple AI models
Running
on
Zero
MCP
2.04k
Z Image Turbo
🖼
2.04k
Generate stunn

In [30]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are a snarkyassistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [24]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [25]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-OCR\nUpdated\n4 days ago\n•\n204k\n•\n743\nmoonshotai/Kimi-K2.5\nUpdated\n2 days ago\n•\n335k\n•\n1.8k\nQwen/Qwen3-Coder-Next\nUpdated\n4 days ago\n•\n53.5k\n•\n544\nstepfun-ai/Step-3.5-Flash\nUpdated\nabout 3 hours ago\n•\n12k\n•\n493\ncirclestone-labs/Anima\nUpdated\n6 days ago\n•\n60.6k\n•\n484\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\n1.28k\nQwen3-TTS Demo\n🎙\

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [26]:
create_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future of machine learning (ML). It is a comprehensive collaboration platform where the global ML community comes together to create, share, and innovate on models, datasets, and applications. Hugging Face empowers ML engineers, scientists, and end users through a vibrant ecosystem and an open-source stack that supports all modalities—text, image, video, audio, and even 3D.

---

## What We Offer

- **Hugging Face Hub**  
  Host and collaborate on unlimited public ML models, datasets, and applications. The hub currently hosts over 2 million models, 500k+ datasets, and 1 million+ AI applications, making it the largest open platform for machine learning.

- **Spaces**  
  A free platform to deploy and access AI apps instantly. Users can explore demos such as text-to-speech, music generation, image creation from text, and animation tools.

- **Open-Source Stack**  
  Hugging Face provides an open-source framework and tools that accelerate ML workflows, enabling users to move faster and be more productive.

- **Enterprise Solutions**  
  Secure, scalable solutions for teams and businesses with enterprise-grade security, access controls, and dedicated compute resources to support AI model development and deployment.

---

## Community & Culture

Hugging Face is more than a platform—it's a thriving community of passionate machine learning professionals and enthusiasts. The company fosters an open and ethical AI future through collaboration and transparency. By sharing work openly and building collective knowledge, the Hugging Face community empowers the next generation of creators, promoting inclusivity and innovation in AI.

With over millions of users engaging weekly, Hugging Face continues to grow as a hub for discovery, experimentation, and professional growth within the AI and ML communities.

---

## Our Customers

Hugging Face serves a broad range of users across academia, startups, and enterprise-level organizations. Customers include:

- Machine learning engineers and researchers building state-of-the-art AI models.
- Data scientists looking for diverse and rich datasets.
- AI application developers seeking easy-to-use AI services.
- Enterprises requiring robust, secure AI infrastructure for team collaboration.

---

## Careers at Hugging Face

Join a mission-driven company at the forefront of AI innovation! At Hugging Face, you will:

- Collaborate with top talents in machine learning and software development.
- Contribute to open-source projects that impact millions worldwide.
- Work in a culture that values openness, ethical AI, transparency, and community-driven growth.

Current openings cover roles in AI research, software engineering, product management, and community engagement. Hugging Face offers an inspiring workplace for those passionate about shaping the future of AI.

---

## Get Started

Explore the future of AI with Hugging Face:

- Browse and use over 2 million machine learning models.
- Access 500k+ datasets to accelerate your projects.
- Deploy or try 1M+ AI-powered applications in Spaces.
- Join the community to learn, share, and build your portfolio.
- Upgrade with paid Compute and Enterprise solutions for professional or team use.

Visit [huggingface.co](https://huggingface.co) to sign up and start building the future of machine learning today!

---

## Brand Identity

- **Colors:**  
  Yellow (#FFD21E), Orange (#FF9D00), Gray (#6B7280)

- **Logo:**  
  Available in PNG, SVG, and AI formats.

---

**Hugging Face** - The AI community building the future.  
Discover. Collaborate. Innovate.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
stream_brochure("HuggingFace", "https://huggingface.co")

# Welcome to Hugging Face: The AI Community That’s Hugging (and Not Poking) the Future

---

## Who Are We?  
Imagine a place where machine learning nerds, data wizards, and AI enthusiasts from all over the globe come together to build, share, and play with AI models, datasets, and applications. That place is **Hugging Face** — and nope, it’s not a cozy therapy group, it’s a *collaboration platform* shaping the future of AI with a jaw-droppingly massive open-source ecosystem.

---

## What’s the Big Deal?  
- **2 Million+ Models**: From OCR to music gen to text-to-speech, our community uploads new models faster than you can say “neural network.”
- **500K+ Datasets**: Yes, half a million—and counting! Don’t worry, it’s all nicely labeled, curated, and ready for your next data obsession.
- **1M+ AI Applications**: Because models are fun, but apps are where the magic happens.
- **Spaces**: Your AI playground, running on anything from “Zero” (free compute) to heavy-hitting GPUs like the A100. Try animating images, synthesizing speech, or generating music—all with just a few clicks.

---

## What Makes Hugging Face So Awesome?  

- **Open Source to the Core**  
  Forget closed doors and paywalls; here, everything is about open collaboration. Want to build your portfolio, share your groundbreaking model, or just snoop on the latest AI trends? You got it.

- **Multi-Modal Madness**  
  Whether it’s text, image, video, audio, or even 3D, Hugging Face speaks your AI language.

- **Enterprise-Grade Muscle**  
  For those who like their AI with a side of security and access controls, Hugging Face offers paid compute and enterprise solutions so your corporate secrets don’t leak like an untrained neural net.

---

## The Culture (AKA Why You’ll Want to Work Here)  
- **Community Fandom**: Our thriving community isn’t just users — it’s a family of machine learning engineers, scientists, and dreamers who want to build an *open and ethical AI future*.  
- **Collaborative Vibes**: Say goodbye to lone wolf coding sessions. At Hugging Face, it’s all about teamwork, shared knowledge, and mutual pats on the back.  
- **Fast and Furious**: We’re constantly updating, innovating, and pushing the envelope—our hub glows with fresh models and datasets daily, just like your favorite binge-worthy series.  
- **Your AI Portfolio Just Got Fancy**: Showcase your work to the world, or heck, impress your future self.

---

## Who Uses Hugging Face?  
- **Machine Learning Pros** seeking the latest open-source models and datasets.  
- **Developers and Researchers** who want to build, deploy, and demo AI without wrangling infrastructure.  
- **Enterprises** craving secure, scalable AI solutions that play nice with existing workflows.  
- **Curious Students and Hobbyists** who just want to dip a toe in the AI pond without drowning.

---

## Careers - Join The AI Revolution  
Interested in creating tools that empower millions of AI enthusiasts worldwide? Hugging Face is hiring smart, passionate people who want to make AI accessible, ethical, and downright fun. If you like working where your code matters, and the community thrives on collaboration—not just competition—then pack your curiosity and apply!

---

## Quick Hugging Face Facts  
- **Brand Colors**: A happy yellow (#FFD21E) and a punchy orange (#FF9D00) to match our upbeat vibe.  
- **Logo**: A smiling face because AI *should* be friendly, not scary.  
- **Community Motto**: Build open, stay ethical, hug the future.

---

### So, why just poke the future when you can give it a big, warm Hugging Face?  

Ready to join us?  
[Sign up](https://huggingface.co/join) — your AI playground awaits.

---  

*Hugging Face: Building AI better, together, one friendly model at a time.*

In [28]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face: Building the Future of AI

---

## Who We Are

Hugging Face is a vibrant, global AI community and collaboration platform dedicated to advancing machine learning (ML). We empower ML engineers, data scientists, and AI enthusiasts to explore, create, and share models, datasets, and applications on an open and ethical AI foundation.

At Hugging Face, we are at the forefront of AI innovation with some of the most widely-used open-source ML libraries and tools, championed by a passionate science team pushing the boundaries of technology.

---

## What We Offer

- **The Hugging Face Hub:**  
  A central repository hosting over 2 million ML models, 500,000+ datasets, and 1 million+ AI applications, supporting modalities including text, image, video, audio, and 3D.

- **Spaces:**  
  An interactive environment to run and demo AI applications, from text-to-speech and music generation to image synthesis and animation.

- **Enterprise Solutions:**  
  Robust AI platform subscriptions starting at $20/user/month with enterprise-grade security, access controls, Single Sign-On (SSO), private dataset support, advanced compute options like ZeroGPU quota boost, analytics dashboards, and centralized token management.

---

## Our Community & Culture

Hugging Face thrives on collaboration and openness. We foster a culture of creativity, inclusivity, and knowledge-sharing where members can build their machine learning portfolios, experiment with state-of-the-art technologies, and contribute to the future of AI.

We hold ourselves to a high ethical standard, striving to create AI solutions that benefit everyone while maintaining transparency and responsibility. Our growing community is featured on popular platforms like GitHub, Twitter, LinkedIn, and Discord, where members actively engage and innovate together.

---

## Customers & Users

Our customers range from individual machine learning practitioners to leading enterprises looking to scale AI across their organizations. We support teams of all sizes with tools for seamless collaboration, security, and accelerated model deployment. Hugging Face’s open-source roots also make us a natural home for researchers, startups, and AI enthusiasts worldwide.

---

## Careers at Hugging Face

Join a fast-growing, mission-driven company at the heart of the AI revolution. We are constantly looking for passionate talent to engage in cutting-edge AI research, engineering, and community building.

Explore current job openings to be part of a diverse team that values innovation, openness, and impact. We offer opportunities to grow professionally while shaping the future of machine learning technology.

---

## Connect with Us

- Website: https://huggingface.co  
- GitHub | Twitter | LinkedIn | Discord  
- Explore models, datasets, and AI applications on the Hugging Face Hub  

---

## Brand Identity

Colors:  
- Yellow: #FFD21E  
- Orange: #FF9D00  
- Gray: #6B7280  

Our logo and brand assets are available for community use, symbolizing our commitment to open collaboration and innovation.

---

**Hugging Face — Accelerate your AI journey with the world’s leading machine learning community and platform.**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>